In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from numpy.linalg import inv
import pandas as pd

In [2]:
x = np.random.rand(1000,3)
x = np.concatenate((x,np.ones((1000,1))), axis=1)
y = np.expand_dims(np.apply_along_axis(lambda x : np.matmul(x,[5,3,1.5,6]), 1, x),axis=1)
x_train, x_test, y_train, y_test = train_test_split(x,y,shuffle=True)

In [3]:
#def accuracy(y,y_predicted):
    #return (y_predicted == y).sum()/len(y)

# As it's linear regression, I'll def
# ine the percentaage error by taking the max abs error and divide it on predicted y then then the accurace will 1 - percentage error
def accuracy(y_test, y_predicted):
    return 1 - (np.max((abs(y_predicted - y_test))/y_predicted))

In [4]:
#Minimizing Sum of Error Squares Estimation by Pseudoinverse
def sum_squared(x,y):
    xT = np.transpose(x) 
    pseudoinverse = inv(np.matrix(np.matmul(xT,x)))
    temp = np.matmul(xT,y)
    weights = np.matmul(pseudoinverse,temp)
    return weights

weights = sum_squared(x_train,y_train)

print("Weights (pseudoinverse): ", np.transpose(weights))

def pseudoinverse_predict(xTrain, yTrain, x):
    w = sum_squared(xTrain, yTrain)
    return np.matmul(x,w);

y_predicted = pseudoinverse_predict(x_train,y_train,x_test)
#print("Accuracy (PseudoInverse): ",accuracy(y_test,y_predicted))

Weights (pseudoinverse):  [[5.  3.  1.5 6. ]]


In [5]:
def Loss_Funtion(y_predicted, y):
    return np.sum(np.square(np.subtract(y,y_predicted)))

In [6]:
def gradient_descent_fit(x, y, learning_rate = 0.0001, iterations = 10000):
    weights = np.random.rand(len(x[0]),1)
    
    for _ in range(iterations):
        gradient = -2 * np.matmul(np.transpose(x), (y - (np.matmul(x, weights))))
        weights = np.subtract(weights, (learning_rate * gradient))
    return weights

def gradient_descent_predict(xTrain,yTrain,x):
    w = gradient_descent_fit(xTrain, yTrain)
    return np.matmul(x, w);

print("Weights (gradient descent): ", np.transpose(gradient_descent_fit(x_train,y_train)))
y_predicted = gradient_descent_predict(x_train,y_train,x_test)
#print("Accuracy (gradient descent): ", accuracy(y_test,y_predicted))

Weights (gradient descent):  [[5.  3.  1.5 6. ]]


In [7]:
#Housing Prices Dataset
hdf = pd.DataFrame(pd.read_csv("USA_Housing.csv"))

maxValue= [hdf["Price"].max(), hdf["Avg. Area Income"].max(), hdf["Avg. Area House Age"].max(), hdf["Avg. Area Number of Rooms"].max(), hdf["Avg. Area Number of Bedrooms"].max(), hdf["Area Population"].max()]

#Normalization to make values between 0 and 1
hdf["Price"] /= maxValue[0]
hdf["Avg. Area Income"] /= maxValue[1]
hdf["Avg. Area House Age"] /= maxValue[2]
hdf["Avg. Area Number of Rooms"] /= maxValue[3]
hdf["Avg. Area Number of Bedrooms"] /= maxValue[4]
hdf["Area Population"] /= maxValue[5]

hdf = hdf.to_numpy()

df_train, df_test = train_test_split(hdf)

housing_Ytrain = (df_train[:,5]).reshape(-1,1)
housing_Xtrain = df_train[:,0:5]
housing_Xtrain = housing_Xtrain.astype('float32')

housing_Ytest = (df_test[:,5]).reshape(-1,1)
housing_Xtest = df_test[:,0:5]
housing_Xtest = housing_Xtest.astype('float32')


#predict using Pseudoinverse
#y_predicted = pseudoinverse_predict(housing_Xtrain, housing_Ytrain, housing_Xtest)
#print("Accuracy HousingPrice (PseudoInverse): ",accuracy(housing_Ytest, y_predicted))

#predict using Gradient Descent
y_predicted = gradient_descent_predict(housing_Xtrain, housing_Ytrain, housing_Xtest)
#print("Accuracy HousingPrice (gradient descent): ",accuracy(housing_Ytest, y_predicted))